In [1]:
import glob
from bddl.activity import Conditions
import pathlib
import re

In [2]:
# Load the combined list
import json
with open(r"C:\Users\cgokmen\Downloads\combined_room_object_list.json", "r") as f:
    scene_room_object_list = json.load(f)["scenes"]

In [3]:
import json, csv
from nltk.corpus import wordnet as wn
import networkx as nx
legit_synsets = {x.name() for x in wn.all_synsets()}

# Build the legit-synset graph
G = nx.DiGraph()
G.add_nodes_from(x.name() for x in wn.all_synsets())
for parent in wn.all_synsets():
    for child in parent.hyponyms():
        G.add_edge(parent.name(), child.name())
        
# Add the illegit-synset graph
with open(r"C:\Users\cgokmen\research\bddl\utils\custom_synsets.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        child = row["custom_synset"].strip()
        parent = wn.synset(row["hypernyms"].strip()).name()
        assert parent in G.nodes, "Could not find " + parent
        G.add_edge(parent, child)

In [4]:
# Get the activity filename map
act_to_fn = {}

from_b1k = glob.glob(r"C:\Users\cgokmen\research\ObjectPropertyAnnotation\init_goal_cond_annotations\problem_files_verified_b1k\*")
from_b100 = glob.glob(r"C:\Users\cgokmen\research\bddl\bddl\activity_definitions\*")
for x in from_b1k + from_b100:
    p = pathlib.Path(x)
    if not p.is_dir():
        continue
    act_name = p.name.replace(" ", "_").replace("-", "_").replace("'", "_")
    if act_name in act_to_fn:
        print(f"Found duplicate for {act_name}")
    act_to_fn[act_name] = p

Found duplicate for packing_car_for_trip


In [5]:
starting_scenes = {"Rs_int", "Beechwood_0_int", "Wainscott_0_int", "Ihlen_0_int", "Merom_1_int", "house_single_floor"}

In [6]:
# The BEHAVIOR 200 activity list
activities_list = """assembling gift baskets
bottling fruit
boxing books up for storage
boxing food after dinner
bringing in kindling
bringing in mail
bringing in wood
bringing newspaper in
bringing paper to recycling
bringing water
brushing lint off clothing
can beans
can_syrup
can_vegetables
canning_food
carrying out garden furniture
carrying water
changing dog's water
chop an onion
chopping vegetables
clean a wood pipe
clean_a_lobster
cleaning a car
cleaning barbecue grill
cleaning bathrooms
cleaning bathtub
cleaning bedroom
cleaning carpets
cleaning closet
cleaning cupboards
cleaning floors
cleaning freezer
cleaning garage
cleaning high chair
cleaning kitchen cupboard
cleaning microwave oven
cleaning out drawers
cleaning oven
cleaning restaurant table
cleaning shoes
cleaning sneakers
cleaning stove
cleaning table after clearing
cleaning the hot tub
cleaning the pool
cleaning toilet
cleaning up after a meal
cleaning up refrigerator
cleaning up the kitchen only
cleaning windows
cleaning_parks
cleaning_the_yard
cleaning_up_plates_and_food
clearing the table after dinner
clearing_table_after_breakfast
clearing_table_after_coffee
clearing_table_after_dinner
clearing_table_after_snacks
clearing_table_after_supper
collect misplaced items
collecting aluminum cans
collecting children's toys
collecting_mail_from_the_letterbox
defrosting freezer
delivering_groceries_to_doorstep
dispose of a pizza box
dispose of batteries
dispose of glass
dispose of medication
dispose of paper
emptying_trash_cans
fill a bucket in a small sink
folding piece of cloth
freeze pies
gathering_nuts
heating food up
heating soup
hiding Easter eggs
installing a fax machine
installing a modem
installing a printer
installing a scanner
installing alarms
laying tile floors
laying wood floors
loading the car
loading the dishwasher
locking every door
locking every window
make a steak
make a strawberry slushie
make_bento
make_gift_bags_for_baby_showers
making tea
mopping floors
moving boxes to storage
mowing the lawn
mulching
opening doors
opening windows
organizing art supplies
organizing boxes in garage
organizing file cabinet
organizing office documents
organizing school stuff
outfit a basic toolbox
pack a beach bag
pack a pencil case
pack your gym bag
packing adult's bags
packing bags or suitcase
packing books into car
packing boxes for household move or trip
packing child's bag
packing cleaning suppies into car
packing documents into car
packing food for work
packing grocery bags into car
packing hiking equipment into car
packing hobby equipment
packing lunches
packing meal for delivery
packing moving van
packing picnic food into car
packing picnic into car
packing sports equipment into car
packing_car_for_trip
packing_recreational_vehicle_for_trip
passing out drinks
picking up litter
picking up take-out food
picking up toys
picking up trash
picking vegetables in garden
place houseplants around your home
polishing furniture
polishing shoes
polishing silver
pour a glass of wine
pouring_water_in_a_glass
praeparing lunch box
prepare a breakfast bar
prepare a slow dinner party
prepare an emergency school kit
prepare and cook prawns
prepare and cook scallops
prepare grapefruit for serving
prepare_a_raised_bed_garden
preparing food for a fundraiser
preserving food
put together a scrapping tool kit
put togethera basic pruning kit
putting away Christmas decorations
putting away games
putting away Halloween decorations
putting away tools
putting away toys
putting backpack in car for school
putting bike in garage
putting dishes away after cleaning
putting leftovers away
putting meal in fridge at work
putting out cat food
putting out dog food
putting shoes on rack
putting up Christmas decorations inside
putting_away_yard_equipment
re-shelving library books
rearranging furniture
recycling glass bottles
recycling newspapers
recycling office papers
serving a meal
serving hors d'oeuvres
set_up_a_coffee_station_in_your_kitchen
setting mousetraps
setting up room for games
setting_up_for_an_event
setting_up_room_for_a_movie
setup a baby crib
setup_a_fish_tank
sorting art supplies
sorting groceries
sorting household items
sorting mail
sorting potatoes
sorting_bottles,_cans,_and_paper
spraying for bugs
stacking wood
stock a bar
store batteries
store beer
store bobby pins
store brownies
store daffodil bulbs
store dahlia bulbs
store eggs
store firewood
store firewood outdoors
store honey
store nuts
store produce
store vodka
store_an_uncooked_turkey
store_silver_coins
store_tulip_bulbs
storing food
storing the groceries
taking trash outside
thaw frozen fish
thawing frozen food
throwing away leftovers
throwing out used napkins
toast coconut
toast sunflower seeds
toast-buns
turn off a normal school calculator
turning on radio
turning sprinkler off
turning sprinkler on
unloading shopping from car
unloading shopping items
unloading the car
unloading_groceries
unpacking suitcase
vacuuming floors
washing cars or other vehicles
washing dishes
washing floor
washing pots and pans
watering houseplants
waxing cars or other vehicles""".replace(" ", "_").replace("-", "_").replace("'", "_").split("\n")
activities = set(activities_list)
# activities = set(act_to_fn.keys())

missing_acts = set(activities) - set(act_to_fn.keys())
print("Total acts", len(activities))
print("Missing acts", len(missing_acts))
assert not missing_acts, "Missing acts: " + str(missing_acts)

Total acts 232
Missing acts 0


In [7]:
scene_mapping = {}
if False:
    with open(r"D:\ig_pipeline\metadata\b1k_activity_to_preselected_scenes.csv", newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row["activity_name"].strip().replace(" ", "_").replace("-", "_").replace("'", "_")
            task_scenes = []
            for i in range(3):
                scene = row[f"scene{i}"].strip()
                if scene:
                    task_scenes.append(scene)
            assert name and task_scenes, f"Failed for {name}"
            scene_mapping[name] = task_scenes
    with open(r"D:\ig_pipeline\metadata\b100_activity_to_preselected_scenes.json") as f:
        mapping = json.load(f)
        for task, scenes in mapping.items():
            name = task.strip().replace(" ", "_").replace("-", "_").replace("'", "_")
            scene_mapping[name] = scenes
    
activities_found_in_mapping = set(scene_mapping.keys())
missing_mappings = activities - activities_found_in_mapping
# assert not missing_mappings, f"Missing mappings: {missing_mappings}"

In [8]:
import collections
from nltk.corpus import wordnet as wn

def counts_for(child, parent):  
    try:
        if nx.has_path(G, parent, child):
            return True
        else:
            c = wn.synset(child).name()
            p = wn.synset(parent).name()
            return nx.has_path(G, p, c)
    except Exception as e:
        # print(e)
        return False

def room_has_requirements(room_contents, room_reqs, verbose=False):
    success = True
    for req_name, req_number in room_reqs.items():
        found_stuff_for_req = 0
        for content_name, content_cnt in room_contents.items():
            if counts_for(content_name, req_name):
                found_stuff_for_req += content_cnt
                
        if found_stuff_for_req < req_number:
            if verbose:
                print(f"            Scene contains {found_stuff_for_req} {req_name} while {req_number} is required.")
            success = False
    return success

def any_room_has_requirements(s, required_rm, required_stuff, verbose=False):
    scene_contents = scene_room_object_list[s]
    matching_scene_rooms = [x for x in scene_contents.keys() if re.fullmatch("^" + required_rm + "_[0-9]+$", x)]
    for msr in matching_scene_rooms:
        if room_has_requirements(scene_contents[msr], required_stuff, verbose=verbose):
            return True
        else:
            if verbose:
                print(f"        {s}'s {msr} is not a match for {required_rm}")

    return False
            
def scene_compatible_with_task(s, task_requirements, verbose=False):
    requirements_by_rm = collections.defaultdict(collections.Counter)
    for obj, rm in task_requirements:
        assert not rm.endswith("_0")
        requirements_by_rm[rm.replace("_0", "")][obj] += 1
    
    for required_rm, required_stuff in requirements_by_rm.items():
        if not any_room_has_requirements(s, required_rm, required_stuff, verbose=verbose):
            if verbose:
                print(f"    Failed in {s} bc we couldn't find a suitable {required_rm}\n")
            return False
        
    return True

def evaluate_task(task_name, believe_assignments=False):
    task_file = act_to_fn[task_name] / "problem0.bddl"
    assert task_file.exists(), f"{task_name} file missing"
    with open(task_file, "r") as f:
        predefined_problem = "".join(f.readlines())
    # for m in re.findall("\(([A-Za-z_]+?) ", predefined_problem):
    #     if m not in pred_wl:
    #         return set()
    # init_states = predefined_problem.split(":init")[1].split(":goal")[0]
    # requirements = list(re.findall(r"\(inroom ([A-Za-z0-9_\.]+)_[0-9]+ ([A-Za-z0-9_\.]+)\)", init_states))
    dom = "omnigibson" if "ObjectPropertyAnnotation" in str(task_file) else "igibson"
    conds = Conditions(task_name, "potato", dom, predefined_problem=predefined_problem)
    objects = set(conds.parsed_objects.keys())
    raw_requirements = [cond[1:] for cond in conds.parsed_initial_conditions if cond[0] == "inroom"]
    assert all(len(req) == 2 for req in raw_requirements), task_name + ":" + str(raw_requirements)
    requirements = [(req.rsplit("_", 1)[0], rm) for req, rm in raw_requirements]
    print(task_name)
    good_scenes = {
        s for s in scene_room_object_list.keys() if s in starting_scenes and scene_compatible_with_task(s, requirements)
    }
    if False:  # task_name in scene_mapping:
        assigned_scenes = set(scene_mapping[task_name]) & starting_scenes
        
        # Temporary forgiveness for things that do match something
        if good_scenes & starting_scenes or not assigned_scenes:
            return objects, good_scenes
        
        missing_assigned_scenes = assigned_scenes - set(good_scenes)
        error_type = "Complete" if missing_assigned_scenes == assigned_scenes else "Partial"
        unexpected_matches = set(good_scenes) - assigned_scenes
        unexpected_match_str = f"Unexpected matches: {unexpected_matches}" if unexpected_matches and missing_assigned_scenes == assigned_scenes else ""
        if missing_assigned_scenes:
            print(f"\n{error_type} mismatch: {task_name} is assigned {assigned_scenes} but {missing_assigned_scenes} do not match init criteria. {unexpected_match_str}")
            for s in missing_assigned_scenes:
                scene_compatible_with_task(s, requirements, verbose=True)
        
        if believe_assignments:
            return objects, good_scenes | assigned_scenes
    return objects, good_scenes

In [9]:
improved_scene_mapping = {x: evaluate_task(x, believe_assignments=False) for x in activities}
unmatched_acts = [act for act, (_, scenes) in improved_scene_mapping.items() if not scenes]
print(len(unmatched_acts))
print(unmatched_acts)

putting_dishes_away_after_cleaning
cleaning_stove
packing_adult_s_bags
picking_up_toys
make_a_steak
cleaning_garage
prepare_an_emergency_school_kit
clearing_table_after_snacks
packing_hobby_equipment
recycling_newspapers
setting_mousetraps
collecting_mail_from_the_letterbox
defrosting_freezer
turning_sprinkler_off
cleaning_up_refrigerator
dispose_of_batteries
make_gift_bags_for_baby_showers
setting_up_room_for_games
cleaning_a_car
rearranging_furniture
bringing_in_kindling
folding_piece_of_cloth
organizing_boxes_in_garage
packing_picnic_food_into_car
serving_hors_d_oeuvres
sorting_art_supplies
locking_every_window
packing_food_for_work
laying_wood_floors
cleaning_up_plates_and_food
installing_a_modem
packing_picnic_into_car
putting_out_dog_food
carrying_water
installing_a_printer
packing_sports_equipment_into_car
store_firewood
thawing_frozen_food
organizing_school_stuff
prepare_and_cook_prawns
putting_meal_in_fridge_at_work
store_tulip_bulbs
packing_grocery_bags_into_car
putting_away_

In [10]:
def get_tasks_possible_with_scenes(scene_set):
    b200_scene_mapping = {x: (improved_scene_mapping[x][0], set(improved_scene_mapping[x][1]) & scene_set) for x in activities if x in improved_scene_mapping}
    return {x: (objects, sorted(scenes)) for x, (objects, scenes) in b200_scene_mapping.items() if scenes}

In [11]:
print(len(get_tasks_possible_with_scenes({"house_single_floor"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Rs_int"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Wainscott_0_int"})))
print(len(get_tasks_possible_with_scenes({"house_single_floor", "Wainscott_0_int", "Rs_int"})))
print(len(get_tasks_possible_with_scenes({"Rs_int", "house_single_floor", "Wainscott_0_int", "Ihlen_0_int"})))
print(len(get_tasks_possible_with_scenes({"Rs_int", "Beechwood_0_int", "Wainscott_0_int", "Ihlen_0_int", "Merom_1_int", "house_single_floor"})))

130
176
185
191
200
214


In [12]:
print("\n".join("1" if (improved_scene_mapping[x][1] - {"house_single_floor"}) else "0" for x in activities_list))

1
1
1
1
0
0
1
0
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
0
0
1
0
0
1
1
0
0
0
0
0
1
0
1
0
1
0
1
0
0
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
1
1
0
1
1
1
0
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1


In [13]:
scene_mapping_repro = {x: scenes for x, (objects, scenes) in get_tasks_possible_with_scenes(starting_scenes).items()}
print("\n".join(str(x) for x in sorted(scene_mapping_repro.items())))

('assembling_gift_baskets', ['Beechwood_0_int', 'Ihlen_0_int', 'Rs_int', 'Wainscott_0_int'])
('bottling_fruit', ['Beechwood_0_int', 'Rs_int', 'Wainscott_0_int'])
('boxing_books_up_for_storage', ['Beechwood_0_int', 'Ihlen_0_int', 'Rs_int'])
('boxing_food_after_dinner', ['Beechwood_0_int', 'Merom_1_int', 'Wainscott_0_int', 'house_single_floor'])
('bringing_in_kindling', ['house_single_floor'])
('bringing_in_mail', ['house_single_floor'])
('bringing_in_wood', ['Beechwood_0_int', 'Merom_1_int', 'Rs_int', 'Wainscott_0_int', 'house_single_floor'])
('bringing_newspaper_in', ['house_single_floor'])
('bringing_paper_to_recycling', ['Beechwood_0_int', 'Ihlen_0_int', 'Merom_1_int', 'Wainscott_0_int', 'house_single_floor'])
('bringing_water', ['house_single_floor'])
('brushing_lint_off_clothing', ['Merom_1_int', 'Rs_int', 'Wainscott_0_int', 'house_single_floor'])
('can_beans', ['Beechwood_0_int', 'Wainscott_0_int'])
('can_syrup', ['Merom_1_int', 'Wainscott_0_int', 'house_single_floor'])
('can_vege

In [14]:
preselection_fn = r"D:\ig_pipeline\metadata\b200_compatible_scenes.json"
with open(preselection_fn, "w") as preselection_f:
    json.dump(scene_mapping_repro, preselection_f, indent=4)
object_mapping = {x: sorted(improved_scene_mapping[x][0]) for x in activities_list}
objects_fn = r"D:\ig_pipeline\metadata\b200_objects.json"
with open(objects_fn, "w") as objects_f:
    json.dump(object_mapping, objects_f, indent=4)

In [ ]:
object_mapping["freeze_pi"]